# Notebook 08 - Overall Quality and Overall Condition data cleaning and fixing

## Objectives
* Clean data
* Evaluate and process missing data
* Fix potential issues with data in feature (OverallCond, OverallQual)

## Inputs
* inputs/datasets/cleaning/masonry_and_porch.csv

## Outputs
* Clean and fix (missing and potentially wrong) data in given column
* After cleaning is completed, we will save current dataset in inputs/datasets/cleaning/clean_finished.csv

## Change working directory
In This section we will get location of current directory and move one step up, to parent folder, so App will be accessing project folder.

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os

current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks/data_cleaning'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("you have set a new current directory")

you have set a new current directory


Confirm new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We need to check current working directory

In [4]:
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We can see that current is **jupyter_notebooks**, as current notebook is in subfolder. We will go one step up to parent directory, what will be our project main directory.
Print out to confirm working directory

In [5]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5'

## Loading Dataset

In [6]:
import pandas as pd

df = pd.read_parquet("inputs/datasets/cleaning/masonry_and_porch.parquet.gzip")
df.head()

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,0,856,854,3,No,706,GLQ,150,0,548,...,65,196,61,5,7,856,0.0,2003,2003,208500
1,1,1262,0,3,Gd,978,ALQ,284,0,460,...,80,0,0,8,6,1262,NaN,1976,1976,181500
2,2,920,866,3,Mn,486,GLQ,434,0,608,...,68,162,42,5,7,920,NaN,2001,2002,223500
3,3,961,0,2,No,216,ALQ,540,0,642,...,60,0,35,5,7,756,NaN,1915,1970,140000
4,4,1145,0,4,Av,655,GLQ,490,0,836,...,84,350,84,5,8,1145,NaN,2000,2000,250000


## Exploring Data

We will get all features that are missing data as a list

In [7]:
print("Overall Condition of house records missing: ", df['OverallCond'].isnull().sum())
print("Overall Quality of house records missing: ", df['OverallQual'].isnull().sum())

Overall Condition of house records missing:  0
Overall Quality of house records missing:  0


We can see that there is no missing data

Now we will check what is data types of these features

In [8]:
df[['OverallCond', 'OverallQual']].dtypes

OverallCond    int64
OverallQual    int64
dtype: object

Both Features have no missing data and are integers.

We will not create one more notebook to check last feature - Wood deck Area, we will include it here
Lets check how many features are missing data

In [9]:
df['WoodDeckSF'].isnull().sum()

1305

We have in total 1305 records with missing data, what means that quantity of buildings that has deck, is very low.
We will replace missing values with 0

In [10]:
df.loc[:, 'WoodDeckSF'] = df['WoodDeckSF'].fillna(value=0)

Let's check what is data type for this feature

In [11]:
df['WoodDeckSF'].dtypes

dtype('float64')

This is float, we will change it to integer

In [12]:
df['WoodDeckSF'] = df['WoodDeckSF'].astype(int)

We have Completed cleaning all dataset.

Now LOUD and PROUD we will check last feature in here - Sales Price. Is there any missing data and what type it is

In [13]:
print("Sales Price is missing data in so many records: ", df['SalePrice'].isnull().sum())
print("Sales Price is this type of data: ", df['SalePrice'].dtypes)

Sales Price is missing data in so many records:  0
Sales Price is this type of data:  int64


Well DONE! All features are valid and correct.

# Exporting dataframe for analysis, modeling, etc

We will export it in inputs/datasets/cleaning/clean_finished.parquet.gzip

In [14]:
df.to_parquet("inputs/datasets/cleaning/clean_finished.parquet.gzip", compression='gzip')

### Splitting data into Train and Test sets and saving them

In [ ]:
import os
from sklearn.model_selection import train_test_split

# Ensure output directories exist
output_dir = 'outputs/datasets/cleaned'
os.makedirs(output_dir, exist_ok=True)  # Creates the directory if it does not exist

# No need to separate features and target just yet, keep the dataframe whole for the split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=13)

# Save the datasets to PARQUET files including 'SalePrice'
df_train.to_parquet(f'{output_dir}/train.parquet.gzip', compression='gzip')
df_test.to_parquet(f'{output_dir}/test.parquet.gzip', compression='gzip')

### Adding Cleaning code to pipeline

```python
# Fill missing values and immediately convert to integers for specified columns
df.loc[:, 'WoodDeckSF'] = df['WoodDeckSF'].fillna(value=0)
df['WoodDeckSF'] = df['WoodDeckSF'].astype(int)
```